# Phase 1: Database Creation
* Take my 9 proxy tickers: **XLK, XLP, XLB, XLF, XLV, XLU, XLI, AGG, AOR**
* Create a 5 year price history of each ticker

### Importance of 'auto_adjust=True'
* Ensures all price columns account for any corporate actions 
* including: dividends, stock splits etc. 

In [5]:
#Importing the yfinance library
import yfinance as yf

proxy_etfs = ["XLK", "XLP", "XLB", "XLF", "XLV", "XLU", "XLI", "AGG"]

#Downloading historical data for each proxy ETF from January 1, 2021 to January 29, 2026
price_data = yf.download(proxy_etfs, start="2005-01-01", end="2026-02-05", auto_adjust=True)

#Extracting the closing prices and saving them to a CSV file
price_table = price_data['Close']
price_table.to_csv("proxy_etf_prices.csv")


[*********************100%***********************]  8 of 8 completed


## Data Transformation
* Convert Prices to Daily Log Returns
* Generate the annualized Covariance Matrix ($\Sigma$)
* Calculate Risk Aversion Coefficient ($\lambda$)

In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("proxy_etf_prices.csv", index_col='Date', parse_dates=True)

#Calculates all the log returns
#Notes:
    #df.shift(1) - finds the previous day's price for each ETF
    #dropna() - removes any rows with NaN values that may result from the shift operation. 
    #This ensures we don't get an error for the first day which doen't have a previous day to compare to.
def log_returns(DataFrame):
    import pandas as pd
    import numpy as np
    log_returns = np.log(DataFrame/DataFrame.shift(1)).dropna()
    log_returns.to_csv("proxy_etf_log_returns.csv")
    return log_returns

log_returns = log_returns(df)

### What different log values mean?
* If $P_t > P_{t-1}$ (Price went up): The ratio is greater than 1, and the log is positive.
* If $P_t < P_{t-1}$ (Price went down): The ratio is between 0 and 1, and the log is negative
* If $P_t = P_{t-1}$ (Price stayed the same): The ratio is exactly 1, and the log is zero.

# Phase 2: Risk Modeling & Implied Returns

Now that you have your table of **Daily Log Returns**, we need to calculate the risk relationships between your 9 sectors and determine what the market "expects" them to return based on your **90/10 benchmark**.

---

## 1. The Annualized Covariance Matrix ($\Sigma$)
The Covariance Matrix is the "heart" of the risk engine. It measures how much each sector deviates from its average and how those deviations move in sync.

### The Math
For any two sectors $i$ and $j$, the covariance is calculated as:
$$\sigma_{i,j} = \frac{\sum_{t=1}^{n} (R_{i,t} - \bar{R}_i)(R_{j,t} - \bar{R}_j)}{n-1}$$

* **Diagonal elements:** Represent the **variance** of each sector.
* **Off-diagonal elements:** Represent the **covariance** between different sectors.

### Annualization
Since your data is daily, but your benchmark is annual, we must scale the matrix:
$$\Sigma_{annual} = \Sigma_{daily} \times 252$$

In [7]:
def cov_matrix(log_returns):
    import pandas as pd
    import numpy as np
    cov_matrix = log_returns.cov() * 252 #Calculates the covariance matrix of the log returns
    return cov_matrix

cov_matrix = cov_matrix(log_returns)

#Adjust display options to see all 9 sectors clearly
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 6)

#Print the table
print("Annualized Covariance Matrix (Sigma):")
print(cov_matrix)

Annualized Covariance Matrix (Sigma):
          AGG       XLB       XLF       XLI       XLK       XLP       XLU       XLV
AGG  0.002750 -0.000143 -0.001483 -0.000395 -0.000154  0.000118  0.001254  0.000219
XLB -0.000143  0.054347  0.050071  0.042012  0.038006  0.021102  0.024813  0.026540
XLF -0.001483  0.050071  0.084903  0.049031  0.044699  0.025471  0.027909  0.030990
XLI -0.000395  0.042012  0.049031  0.043538  0.036635  0.020438  0.023088  0.025333
XLK -0.000154  0.038006  0.044699  0.036635  0.050154  0.019638  0.021707  0.025502
XLP  0.000118  0.021102  0.025471  0.020438  0.019638  0.019910  0.018166  0.016946
XLU  0.001254  0.024813  0.027909  0.023088  0.021707  0.018166  0.034744  0.018480
XLV  0.000219  0.026540  0.030990  0.025333  0.025502  0.016946  0.018480  0.027791


# Phase 3: Establishing the Market Baseline

Now that you have your **Annualized Covariance Matrix ($\Sigma$)**, we need to calculate the "Market Price of Risk" and the returns the market implies for your 9 sectors under a **90/10 benchmark**.

---

## Step 2: Calculate Risk Aversion ($\lambda$)

The Risk Aversion Coefficient ($\lambda$) represents the trade-off between risk and return for the entire market. It tells the model how much extra return is required for every additional unit of variance.

### The Math
$$\lambda = \frac{E(R_m) - R_f}{\sigma^2_m}$$

* **$E(R_m)$**: The expected annual return of your 90/10 benchmark (e.g., 0.08 for 8%).
* **$R_f$**: The Risk-Free Rate (typically the yield on the 10-year U.S. Treasury, e.g., 0.042 for 4.2%).
* **$\sigma^2_m$**: The variance of the benchmark, calculated as $w_{mkt}^T \Sigma w_{mkt}$.

In [13]:
def eighty_twenty(cov_matrix):
    import pandas as pd
    #Define weights with explicit Ticker labels
    weights_dict = {
        'XLK': 0.21, 'XLP': 0.11, 'XLB': 0.11, 'XLF': 0.11, 
        'XLV': 0.11, 'XLU': 0.04, 'XLI': 0.11, 'AGG': 0.20
    }
    #Create a Series and reindex it to match the Covariance Matrix exactly
    w_series = pd.Series(weights_dict).reindex(cov_matrix.index)
    return w_series

def sixty_fourty(cov_matrix):
    import pandas as pd
    #Define weights with explicit Ticker labels
    weights_dict = {
        'XLK': 0.17, 'XLP': 0.09, 'XLB': 0.08, 'XLF': 0.08, 
        'XLV': 0.08, 'XLU': 0.04, 'XLI': 0.09, 'AGG': 0.40
    }
    #Create a Series and reindex it to match the Covariance Matrix exactly
    w_series = pd.Series(weights_dict).reindex(cov_matrix.index)
    return w_series

def ninety_ten(cov_matrix):
    import pandas as pd
    #Define weights with explicit Ticker labels
    weights_dict = {
        'XLK': 0.24, 'XLP': 0.12, 'XLB': 0.12, 'XLF': 0.12, 
        'XLV': 0.12, 'XLU': 0.06, 'XLI': 0.12, 'AGG': 0.10
    }
    #Create a Series and reindex it to match the Covariance Matrix exactly
    w_series = pd.Series(weights_dict).reindex(cov_matrix.index)
    return w_series

#Fix WEIGHTS
#def simm_bench(cov_matrix):
#    import pandas as pd
    # Define weights with explicit Ticker labels
#    weights_dict = {
#        'XLK': 0.18, 'XLP': 0.09, 'XLB': 0.09, 'XLF': 0.09, 
#        'XLV': 0.09, 'XLU': 0.04, 'XLI': 0.09, 'AGG': 0.20, 'AOR': 0.13
#    }
    # Create a Series and reindex it to match the Covariance Matrix exactly
#    w_series = pd.Series(weights_dict).reindex(cov_matrix.index)
#    return w_series

#Calculates the Benchmark variance
def benchmark_variance(market_series, covariance_matrix):
    return market_series.T @ covariance_matrix @ market_series

#Using an 80/20 market baseline
var = benchmark_variance(eighty_twenty(cov_matrix), cov_matrix)
print(f"Benchmark Variance: {var:.6f}")
print(f"Benchamrk Standard Deviation: {np.sqrt(var):.6f}")

#Calculation to find lambda
def lambda_risk_aversion(benchmark_variance, benchmark):
    E_RM = 0;
    if benchmark == "90/10": E_RM = 0.09
    elif benchmark == "80/20": E_RM = 0.085
    else: E_RM = 0.0725
    RF = 0.0421
    
    return (E_RM-RF)/benchmark_variance


print(f"Risk Aversion (Lambda): {lambda_risk_aversion(var, '80/20'):.6f}")

Benchmark Variance: 0.021505
Benchamrk Standard Deviation: 0.146644
Risk Aversion (Lambda): 1.994926


| Value of $\lambda$ | Sentiment | Portfolio Behavior |
| :--- | :--- | :--- |
| **High $\lambda$** (e.g., 3.5+) | **Risk Averse** | The model will prefer "safe" sectors like Fixed Income and Utilities. It hates volatility and will sacrifice returns to avoid it. |
| **Low $\lambda$** (e.g., 1.5 - 2.5) | **Risk Tolerant** | The model is "braver." It will aggressively allocate to high-volatility sectors like Technology and Industrials to capture higher growth. |

# Phase 3: Step 3 — Implied Equilibrium Returns (Π)

This step is known as **Reverse Optimization**. Instead of trying to find the best weights for a set of returns, we assume that your **90/10 benchmark weights** are already optimal and calculate the returns that would justify them.

### The Mathematical Formula
$$\Pi = \lambda \Sigma w_{mkt}$$

* **$\Pi$**: A vector containing the "neutral" expected returns for each of your 9 sectors.
* **$\lambda$**: The Risk Aversion Coefficient (calculated in Step 2).
* **$\Sigma$**: Your $9 \times 9$ Annualized Covariance Matrix (calculated in Step 1).
* **$w_{mkt}$**: Your benchmark weight vector (e.g., 0.10 for Fixed Income, and the remaining 0.90 distributed across equities).


In [14]:
#Calculate the implied Equilubrium returns 
#'@' performs matrix multiplication

def implied_returns(lambda_risk_aversion, cov_matrix, market):
    import numpy as np
    import pandas as pd
    return lambda_risk_aversion * cov_matrix @ market
    
ir = implied_returns(lambda_risk_aversion(benchmark_variance(eighty_twenty(cov_matrix), cov_matrix), "80/20"), cov_matrix, eighty_twenty(cov_matrix))

#Create a DataFrame for better readability
implied_df = pd.DataFrame(
    ir, 
    index=log_returns.columns, #Use the column names from your log_returns to label the sectors
    columns=['Implied Equilibrium Return']
)

#Format as percentages for readability
implied_df['Implied Equilibrium Return'] = implied_df['Implied Equilibrium Return'].map(lambda x: f'{x:.2%}')

print("--- Step 3: Implied Equilibrium Returns (Pi) ---")
print(implied_df)

--- Step 3: Implied Equilibrium Returns (Pi) ---
    Implied Equilibrium Return
AGG                      0.08%
XLB                      6.04%
XLF                      7.31%
XLI                      5.66%
XLK                      5.88%
XLP                      3.25%
XLU                      3.70%
XLV                      4.02%


In [15]:

#Defining my Sector Order
sectors = list(log_returns.columns)
print("Sector Order:", sectors)

Sector Order: ['AGG', 'XLB', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV']


In [16]:
import numpy as np
import pandas as pd

#Define my Analyst Views
#Relative views include a 'subsidiary' (the loser)
#Absolute views set 'subsidiary' to None
views_list = [
    {'target': 'XLB', 'subsidiary': None, 'return': 0.05},  #Absolute
    {'target': 'XLI', 'subsidiary': None, 'return': 0.06}, #Absolute
    {'target': 'XLV', 'subsidiary': None, 'return': 0.035}, #Absolute
    {'target': 'XLF', 'subsidiary': 'XLP', 'return': 0.02}  #Relative (XLF > XLP)
]

def view_vectors(views, sectors):
    import numpy as np
    import pandas as pd
    num_views = len(views)
    num_assets = len(sectors)
    
    #Initialize P and Q
    P = np.zeros((num_views, num_assets))
    Q = np.zeros(num_views)
    
    for i, view in enumerate(views):
        #Handle the Pick Matrix (P)
        #Set the target (Winner)
        P[i, sectors.index(view['target'])] = 1
        
        # et the subsidiary (Loser) if it exists
        if view['subsidiary']:
            P[i, sectors.index(view['subsidiary'])] = -1
            
        #Handle the View Vector (Q)
        #This always takes the expected return value
        Q[i] = view['return']
        
    return P, Q

P, Q = view_vectors(views_list, sectors)

print("\n--- Pick Matrix (P) ---")
print(pd.DataFrame(P, columns=sectors))
print("\n--- View Vector (Q) ---")
print(Q)


--- Pick Matrix (P) ---
   AGG  XLB  XLF  XLI  XLK  XLP  XLU  XLV
0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0
3  0.0  0.0  1.0  0.0  0.0 -1.0  0.0  0.0

--- View Vector (Q) ---
[0.05  0.06  0.035 0.02 ]


### **Think of it this way:** 
$P$ and $Q$ tell the model what you think; $\tau$ and $\Omega$ tell the model how much to care about what you think.
* $\tau$ represents how much you trust the "Market Baseline" ($\Pi$) relative to your own data. (Note 0.025 is the standard)
* $\Omega$ is the certainty of the analysts views

I will use the The He-Litterman Method to find my omega.

This method takes my Covariance Matrix ($\Sigma$), and assumes our confidence in our views is proportional to the corresponding  sector's volatility.

In [17]:
#Calculate Omega (The uncertainty of the analyst views)
#This creates a diagonal matrix representing the variance of your views
#'He-Litterman' method: Omega = diag(P * (tau * Sigma) * P.T)

def omega(P_matrix, covariance_matrix):
    import numpy as np
    TAU = 0.025
    
    omega = np.diag(np.diag(P_matrix @ (TAU * covariance_matrix) @ P_matrix.T))
    return omega

print("Uncertainty Matrix (Omega):")
omega = omega(P, cov_matrix)
print(omega)

Uncertainty Matrix (Omega):
[[0.00135867 0.         0.         0.        ]
 [0.         0.00108844 0.         0.        ]
 [0.         0.         0.00069477 0.        ]
 [0.         0.         0.         0.00134676]]


In [18]:
def mu_bl(cov_matrix, P_matrix, Q_matrix, omega, implied_returns):
    TAU = 0.025
    
    #Calculate the 'Prior' precision (Tau * Sigma)^-1
    precision_prior = np.linalg.inv(TAU * cov_matrix)

    #Calculate the 'View' precision (P.T * Omega^-1 * P)
    precision_view = P_matrix.T @ np.linalg.inv(omega) @ P_matrix

    #Calculate the combined returns (Mu_BL)
    #Formula: [(Prior_Prec + View_Prec)^-1] @ [Prior_Prec @ Pi + P.T @ Omega^-1 @ Q]
    term1 = np.linalg.inv(precision_prior + precision_view)
    term2 = (precision_prior @ implied_returns) + (P_matrix.T @ np.linalg.inv(omega) @ Q_matrix)
    
    mu_bl = term1 @ term2
    return mu_bl
mu_bl = mu_bl(cov_matrix, P, Q, omega, ir)

## **Final Phase:**
### **Running the Black-Litterman Utility Formula**
The Master Formula:$$\mu_{BL} = [(\tau \Sigma)^{-1} + P^T \Omega^{-1} P]^{-1} [(\tau \Sigma)^{-1} \Pi + P^T \Omega^{-1} Q]$$

In [19]:
#Define the Sector Mapping (Matches your 9 sectors)
sector_names = {
    'XLK': 'Technology',
    'XLP': 'Consumer Staples',
    'XLB': 'Materials',
    'XLF': 'Financials',
    'XLV': 'Healthcare',
    'XLU': 'Utilities',
    'XLI': 'Industrials',
    'AGG': 'Fixed Income',
}

#Correct the View Mapping 
#This ensures analyst views appear in the right row even if you only have one view
view_dict = {}
for i in range(len(Q)):
    #Find which sector index has the '1' in this row of P
    sector_idx = np.where(P[i] == 1)[0][0]
    view_dict[sectors[sector_idx]] = Q[i]

#Create the comparison DataFrame
comparison_df = pd.DataFrame({
    'Sector Name': [sector_names.get(s, s) for s in sectors], #Mapping tickers to Names
    'Market Implied (Pi)': ir,
    'Analyst View (Q)': [view_dict.get(s, np.nan) for s in sectors],
    'BL Combined (Mu)': mu_bl
}, index=sectors)

#Add the 'Total' Return (Adding Risk-Free Rate back)
comparison_df['Total Return (Exp)'] = comparison_df['BL Combined (Mu)'] + 0.0421 #risk free rate

print("--- Final Black-Litterman Portfolio Analysis ---")

#Apply formatting
def format_perc(val):
    if isinstance(val, str): return val
    if pd.isna(val): return "-"
    return f"{val:.2%}"
#tries .map first and if it is the wrong version it will use .applymap
try:
    print(comparison_df.map(format_perc))
except AttributeError:
    print(comparison_df.applymap(format_perc))

--- Final Black-Litterman Portfolio Analysis ---
          Sector Name Market Implied (Pi) Analyst View (Q) BL Combined (Mu) Total Return (Exp)
AGG      Fixed Income               0.08%                -            0.10%              4.31%
XLB         Materials               6.04%            5.00%            5.48%              9.69%
XLF        Financials               7.31%            2.00%            6.26%             10.47%
XLI       Industrials               5.66%            6.00%            5.25%              9.46%
XLK        Technology               5.88%                -            5.46%              9.67%
XLP  Consumer Staples               3.25%                -            3.17%              7.38%
XLU         Utilities               3.70%                -            3.54%              7.75%
XLV        Healthcare               4.02%            3.50%            3.72%              7.93%


## Reading the results:
* ### If $\mu_{BL}$ is higher than the market implied return , you will increase your position from the current weight.
* ### If $\mu_{BL}$ is lower for Consumer Staples (XLP), you will harvest those gains and move them elsewhere.

## Interpreting the Model
This model tells us how to adjust sector weights based on our views assuming they are at the market baseline given. So if the model shows that industrials is undervalued and its weight should be increased. It is saying increase the weight from the current benchmark you inputted. 

First look at your portfolio because you may already be sitting above the benchamrk
